In [1]:
from glob import glob
from itertools import (islice, zip_longest)
from collections import Counter
import numpy as np
from sys import argv

import raw_taq

import importlib
importlib.reload(raw_taq)

<module 'raw_taq' from 'c:\\Users\\jie\\Documents\\GitHub\\dlab-finance\\basic-taq\\raw_taq.py'>

In [2]:
def count_chunk_elements(fname, chunksize = 1000000, max_chunk = None, process_chunk = True):

    symbol_roots = Counter()

    for (i,chunk) in enumerate(islice(raw_taq.TAQ2Chunks(fname, chunksize=chunksize, do_process_chunk=process_chunk), max_chunk)):

        counts = np.unique(chunk[:]['Symbol_root'], return_counts=True)
        symbol_roots.update(dict(zip_longest(counts[0], counts[1])))

        #print("\r {0}".format(i),end="")

    return symbol_roots

In [3]:
#fname = '../local_data/EQY_US_ALL_BBO_20150102.zip'
fname = '../local_data/EQY_US_ALL_BBO_20140206.zip'


In [12]:
%time c = count_chunk_elements(fname, chunksize = 1000, max_chunk=None, process_chunk = False)


Wall time: 4min 30s


In [10]:
# The TAQ files are in ASCII, not utf-8. 
# It won't make a difference, but creates incorrect assumptions for later, maybe!
# Also, using nested unpacking in for loops is discouraged, as it's hard to follow. -DJC
for (i,(k,v)) in enumerate(islice(c.most_common(),100)):
    print ("\t".join([str(i), k.decode('utf-8').strip(), str(v)]))

0	SPY	10297058
1	QQQ	5740732
2	DIA	5611075
3	IWM	5467710
4	VXX	4312499
5	UVXY	3682506
6	SSO	3235515
7	XIV	3229013
8	IVV	2922134
9	TNA	2504003
10	GM	2452497
11	SDS	2324211
12	UNG	2264448
13	QLD	2250398
14	UGAZ	2193335
15	EEM	2179802
16	SPXU	2058427
17	UPRO	2005122
18	TZA	2002577
19	XLE	1986953
20	EFA	1935417
21	QID	1845243
22	SVXY	1825746
23	VIIX	1798330
24	C	1783541
25	VALE	1721815
26	SPXL	1657774
27	JPM	1653048
28	FB	1615823
29	SQQQ	1605698
30	SDOW	1572361
31	PBR	1541600
32	VOO	1534326
33	MSFT	1517661
34	DDM	1496905
35	XLI	1486355
36	XLY	1482653
37	BAC	1458931
38	CHK	1421841
39	EWZ	1414226
40	VZ	1411516
41	MU	1406021
42	GE	1398166
43	WFC	1385294
44	TQQQ	1299812
45	GLD	1286694
46	UDOW	1286300
47	IYR	1259243
48	MDY	1248677
49	PFE	1232134
50	FAS	1206044
51	TWM	1192135
52	XLB	1181770
53	PNC	1168967
54	CVX	1149495
55	XLV	1146126
56	DHI	1145891
57	VOD	1131235
58	CMCS	1128529
59	FAZ	1124740
60	T	1118109
61	DXD	1110021
62	XHB	1079966
63	MS	1058689
64	EMC	1056158
65	VWO	1043193
66	F	1031661
67

Converting to HDF5


In [1]:
import numpy as np
import tables as tb

import raw_taq
from importlib import reload
reload(raw_taq)

<module 'raw_taq' from 'c:\\Users\\jie\\Documents\\GitHub\\dlab-finance\\basic-taq\\raw_taq.py'>

In [2]:
test = raw_taq.TAQ2Chunks('../local_data/EQY_US_ALL_BBO_20140206.zip', do_process_chunk=True)

In [3]:
h5 = test.setup_hdf5('20140206')

In [4]:
%time chunk = next(test.iter_)

print( chunk )

Wall time: 4.98 s
[ (1391677200.901, 4, 0, 901, b'P', b'A     ', b'          ', 0.0, 0, 0.0, 0, b'R', b'    ', b'P', b'P', 14, b'2', b'2', b' ', b'C', b' ', b' ', b' ', b' ', b' ', b' ', b' ')
 (1391688108.081, 7, 53, 81, b'P', b'A     ', b'          ', 0.0, 0, 72.94, 27, b'R', b'    ', b'P', b'P', 76255, b'1', b'2', b' ', b'C', b' ', b' ', b' ', b' ', b' ', b' ', b' ')
 (1391688108.085, 7, 53, 85, b'P', b'A     ', b'          ', 0.0, 0, 60.76, 10, b'R', b'    ', b'P', b'P', 76256, b'1', b'2', b' ', b'C', b' ', b' ', b' ', b' ', b' ', b' ', b' ')
 ...,
 (1391633315.79, 10, 30, 43790, b'P', b'AAL   ', b'          ', 34.51, 1, 34.58, 6, b'R', b'    ', b'P', b'P', 3798327, b' ', b'0', b' ', b'N', b' ', b' ', b' ', b' ', b' ', b' ', b'A')
 (1391633315.79, 10, 30, 43790, b'C', b'AAL   ', b'          ', 0.0, 0, 34.56, 1, b'R', b'    ', b'C', b'C', 3798328, b' ', b'2', b' ', b'N', b' ', b' ', b' ', b' ', b' ', b' ', b'A')
 (1391633315.791, 10, 30, 43791, b'P', b'AAL   ', b'          ', 34.51,

In [ ]:
h5.append( chunk ) #does not work from here, python crash

In [1]:
test.finalize_hdf5()

h5 = tb.open_file('./20140206.h5')
h5.root.daily_quotes[:]

NameError: name 'test' is not defined

Using raw_taq inside function

In [1]:
import numpy as np
import tables as tb

import raw_taq
from importlib import reload
reload(raw_taq)

test = raw_taq.TAQ2Chunks('../local_data/EQY_US_ALL_BBO_20140206.zip', do_process_chunk=True)

In [ ]:
h5 = test.to_hdf5() #python also crash

runing h5 outside function

In [1]:
import numpy as np
import tables as tb
import raw_taq

from importlib import reload
reload(raw_taq)

<module 'raw_taq' from 'c:\\Users\\jie\\Documents\\GitHub\\dlab-finance\\basic-taq\\raw_taq.py'>

In [2]:
fname = '../local_data/EQY_US_ALL_BBO_20140206.zip'
test = raw_taq.TAQ2Chunks(fname,chunksize=10, do_process_chunk=True)

In [3]:
h5 = tb.open_file('20140206.h5', title='TAQ',mode='w', filters=tb.Filters(complevel=8, complib='blosc:lz4hc', fletcher32=True) )
table = h5.create_table('/', 'daily_quotes', description = test.bytes_spec.pytables_desc, expectedrows=test.numlines)

table.append(next(test.iter_)) #worked, and create a 20140206.h5 file


/daily_quotes (Table(0,), fletcher32, shuffle, blosc:lz4hc(8)) ''


In [4]:
h5 = tb.open_file('./20140206.h5') #read the first chunk
h5.root.daily_quotes[:]

array([ (1391677200.901, 4, 0, 901, b'P', b'A     ', b'          ', 0.0, 0, 0.0, 0, b'R', b'    ', b'P', b'P', 14, b'2', b'2', b' ', b'C', b' ', b' ', b' ', b' ', b' ', b' ', b' '),
       (1391688108.081, 7, 53, 81, b'P', b'A     ', b'          ', 0.0, 0, 72.94, 27, b'R', b'    ', b'P', b'P', 76255, b'1', b'2', b' ', b'C', b' ', b' ', b' ', b' ', b' ', b' ', b' '),
       (1391688108.085, 7, 53, 85, b'P', b'A     ', b'          ', 0.0, 0, 60.76, 10, b'R', b'    ', b'P', b'P', 76256, b'1', b'2', b' ', b'C', b' ', b' ', b' ', b' ', b' ', b' ', b' '),
       (1391688108.089, 7, 53, 89, b'P', b'A     ', b'          ', 41.9, 27, 60.76, 10, b'R', b'    ', b'P', b'P', 76257, b'1', b'2', b' ', b'C', b' ', b' ', b' ', b' ', b' ', b' ', b' '),
       (1391688108.094, 7, 53, 94, b'P', b'A     ', b'          ', 54.07, 27, 60.76, 10, b'R', b'    ', b'P', b'P', 76258, b'1', b'2', b' ', b'C', b' ', b' ', b' ', b' ', b' ', b' ', b' '),
       (1391688036.249, 7, 56, 4249, b'P', b'A     ', b'         

In [ ]:
h5.close()